<a href="https://colab.research.google.com/github/TechWizard101/GDDA612/blob/main/GDDA708_W4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
ls

drive/  sample_data/


In [7]:
cd drive

/content/drive


In [8]:
ls

MyDrive/


In [9]:
cd MyDrive

/content/drive/MyDrive


In [10]:
ls

 BL-Flickr-Images-Book.csv  'Colab Notebooks'/  'GDDA7124C - Data Analytics'/   olympics.csv


In [11]:
cd GDDA7124C - Data Analytics

/content/drive/MyDrive/GDDA7124C - Data Analytics


In [12]:
ls

 BL-Flickr-Images-Book.csv   heart.csv   olympics.csv               'train .txt.csv'
 GDDA708_Lab03_DTree.ipynb   hw1.ipynb   olympics.ipynb             'Week 3 - olympics.ipynb'
 GDDA708_W4.ipynb            Iris.xlsx   pima-indians-diabetes.csv


In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, f_classif

In [14]:
# Load the data
rawdata = pd.read_csv('pima-indians-diabetes.csv')

# Separate features (X) and target variable (Y)
X = rawdata.drop(columns=['Class'])  # Assuming 'Class' is the target variable
Y = rawdata['Class']

In [15]:
#2)	Feature Extraction with Univariate Statistical Tests
# Initialize SelectKBest with f_classif scoring and k=3 (top 3 features)
feature_selector = SelectKBest(score_func=f_classif, k=3)

# Fit SelectKBest to the data
fit = feature_selector.fit(X, Y)

# Get the indices of the selected features
selected_features_indices = fit.get_support(indices=True)

In [16]:
# Get the scores of the features
feature_scores = fit.scores_

# Create a DataFrame to summarize scores with feature names
score_summary = pd.DataFrame({'Feature': X.columns, 'Score': feature_scores})

# Sort the DataFrame by scores in descending order
score_summary = score_summary.sort_values(by='Score', ascending=False)

# Print the summary
print("Feature scores:")
print(score_summary)


Feature scores:
                    Feature       Score
1                   Glucose  213.161752
5                       BMI   71.772072
7                       Age   46.140611
0               Pregnancies   39.670227
6  DiabetesPedigreeFunction   23.871300
4                   Insulin   13.281108
3             SkinThickness    4.304381
2             BloodPressure    3.256950


In [17]:
# Get the names of the selected features
selected_feature_names = X.columns[selected_features_indices]

# Print the names of the selected features
print("Names of the selected features:")
for feature_name in selected_feature_names:
    print(feature_name)

Names of the selected features:
Glucose
BMI
Age


In [18]:
# Get the indices of the K most significant features
k_most_significant_indices = selected_features_indices[:3]  # Replace K with the desired number of features

# Subset the feature matrix with the K most significant features
X_selected = X.iloc[:, k_most_significant_indices]

# Print the shape of the new feature matrix
print("Shape of the selected feature matrix:", X_selected.shape)


Shape of the selected feature matrix: (768, 3)


In [19]:
# Split the data into training and testing sets
pred_train, pred_test, tar_train, tar_test = train_test_split(X_selected, Y, test_size=0.3, random_state=2)

# Initialize the logistic regression model
logreg = LogisticRegression(random_state=2)

# Fit the model to the training data
logreg.fit(pred_train, tar_train)

# Make predictions on the testing data
predictions = logreg.predict(pred_test)

# Compute accuracy score
accuracy = accuracy_score(tar_test, predictions)

# Print the accuracy score
print("Accuracy score of the logistic regression model: %.2f" % accuracy)


Accuracy score of the logistic regression model: 0.77


In [20]:
'''
3)	Feature Extraction with - Recursive Feature Extraction (RFE)

'''
# Initialize logistic regression model
logreg = LogisticRegression(max_iter=1000)

# Initialize RFE with the logistic regression model and the desired number of features
rfe = RFE(estimator=logreg, n_features_to_select=3)

# Fit RFE to the data
rfe.fit(X, Y)

# Get the indices of the selected features
selected_features_indices = rfe.support_

# Get the names of the selected features
selected_feature_names = [f"Feature {i+1}" for i, selected in enumerate(selected_features_indices) if selected]


In [21]:
 #Get the indices of the selected features
selected_features_indices = rfe.support_
# Get the names of the selected features
selected_feature_names = X.columns[selected_features_indices]

In [22]:
# Print the number of features selected
print("Num Features: %d" % rfe.n_features_)
# Print the names of the selected features
print("Names of the selected features:")
for feature_name in selected_feature_names:
    print(feature_name)
# Print the feature ranking
print("Feature Ranking: %s" % rfe.ranking_)

Num Features: 3
Names of the selected features:
Pregnancies
BMI
DiabetesPedigreeFunction
Feature Ranking: [1 2 4 6 5 1 1 3]


In [23]:
# Transform the original feature matrix to retain only the selected features
selected_features = rfe.transform(X)
# Extract the indices of the selected features
selected_features_indices = rfe.get_support(indices=True)

In [24]:
selected_features_indices = selected_features_indices[:3]

In [25]:
# Select the K most significant features if there are at least K features selected
K = 3  # Replace with the desired number of features
if len(selected_features_indices) >= K:
    selected_features_indices = selected_features_indices[:K]
    pred_features = selected_features
else:
    print("Number of features selected by RFE is less than K.")

# Split the data into training and testing sets
pred_train, pred_test, tar_train, tar_test = train_test_split(pred_features, Y, test_size=0.3, random_state=20)

In [26]:
# Train a logistic regression model using the selected features
logreg_rfe = LogisticRegression(max_iter=1000)
logreg_rfe.fit(pred_train, tar_train)

# Make predictions on the testing data
predictions_rfe = logreg_rfe.predict(pred_test)

# Compute accuracy score
accuracy_rfe = accuracy_score(tar_test, predictions_rfe)

# Print the accuracy score
print("Accuracy score of the model with RFE selection: %.2f" % accuracy_rfe)


Accuracy score of the model with RFE selection: 0.68


In [27]:

# Initialize PCA with the desired number of components (K=3)
pca = PCA(n_components=3)

# Fit PCA to the data
fit = pca.fit(X)

# Summarize components
print("Explained Variance: %s" % (fit.explained_variance_ratio_))
print("Components:\n", fit.components_)

# Apply only the K most significant features (components) according to PCA feature selection
features = fit.transform(X)
pred_features = features[:, :3]

# Split the data into training and testing sets
pred_train, pred_test, tar_train, tar_test = train_test_split(pred_features, Y, test_size=0.3, random_state=2)




Explained Variance: [0.88854663 0.06159078 0.02579012]
Components:
 [[-2.02176587e-03  9.78115765e-02  1.60930503e-02  6.07566861e-02
   9.93110844e-01  1.40108085e-02  5.37167919e-04 -3.56474430e-03]
 [-2.26488861e-02 -9.72210040e-01 -1.41909330e-01  5.78614699e-02
   9.46266913e-02 -4.69729766e-02 -8.16804621e-04 -1.40168181e-01]
 [-2.24649003e-02  1.43428710e-01 -9.22467192e-01 -3.07013055e-01
   2.09773019e-02 -1.32444542e-01 -6.39983017e-04 -1.25454310e-01]]


In [28]:
# Initialize logistic regression model
logreg_pca = LogisticRegression(max_iter=1000)

# Fit logistic regression model to the PCA-transformed features
logreg_pca.fit(pred_train, tar_train)

# Make predictions on the test set
predictions_pca = logreg_pca.predict(pred_test)

# Compute accuracy score
accuracy_pca = accuracy_score(tar_test, predictions_pca)

# Print the accuracy score
print("Accuracy score of our model with PCA selection and logistic regression: %.2f" % accuracy_pca)

Accuracy score of our model with PCA selection and logistic regression: 0.75


In [29]:
from scipy.stats import skew, kurtosis
# Calculate skewness
skewness = skew(rawdata)
print("Skewness:", skewness)


Skewness: [ 0.89991194  0.17341396 -1.84000523  0.10915876  2.26781046 -0.42814328
  1.9161592   1.12738926  0.6337757 ]
